# hVAE

In [2]:
from src.hvae import hVAE
from hparams import get_hparams

PATH = "/Users/laszlofreund/code/ai/hvae/experiments/TDVAE/2023-08-27__23-49/checkpoints/checkpoint-0.pth"
#dataset = get_hparams().data_params.dataset
#test_loader = dataset.get_test_loader()

hvae = hVAE.load(PATH)
#hvae.test_model(test_loader)
hvae.summary()

FileNotFoundError: [Errno 2] No such file or directory: 'data/textures/datasets'